In [ ]:
# Python 2.x / 3.x compatibility
from __future__ import division, print_function

In [ ]:
%matplotlib inline

#Import modules
import pandas as pd
import numpy as np
import os
import json
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import re

#import csv
import glob as gb

#import pathlib

import datetime
import sqlite3

from df2gspread import df2gspread as d2g

mpl.rcParams['figure.figsize'] = (16, 9)
pd.options.display.max_rows = 200

In [ ]:
from python import regression_analysis
# from imp import reload
# reload(regression_analysis)

from python.regression_analysis import background_colors, getStyles
styles = getStyles()

# def background_colors(val):
#     fmt = ''
#     s = 'background-color: {}'
#     if val == 'Fail':
#         fmt = s.format('#F4C7C3')
#     elif val == 'N/A':
#         fmt = s.format('#EDEDED') +"; color: #ADADAD;"
#     elif val == '':
#         fmt = s.format('#f2e2c1')
#     return fmt

# def hover(hover_color="#ffff99"):
#     return dict(selector="tr:hover",
#                 props=[("background-color", "%s" % hover_color)])

# styles = [
#     hover(),
#     dict(selector="td", props=[#("font-size", "150%"),
#                                ("text-align", "center")]),
#     dict(selector="caption", props=[("caption-side", "bottom")])
# ]

In [ ]:
ROOT_DIR = os.getcwd()
ROOT_DIR

# Parse compatibility Matrix

In [ ]:
compat_matrix = regression_analysis.parse_compatibility_matrix()
compat_matrix.head()

## Forcing a new version

The compatibility matrix is parsed from its [online location](https://github.com/NREL/OpenStudio/wiki/OpenStudio-Version-Compatibility-Matrix) and is used for looking up E+ versions corresponding to your OpenStudio version.

If you are working on a custom develop local build, you don't want to be prompted each time you parse df_files, so you can directly add it here

**If you want to get info from your local build**


In [ ]:
version_info = regression_analysis.test_os_cli('/home/julien/Software/Others/OS-build-release/Products/openstudio')
# version_info = regression_analysis.test_os_cli('openstudio')

In [ ]:
version_info

In [ ]:
version = '.'.join([version_info['major'], version_info['minor'], version_info['patch']])
if version_info['prerelease']:
    version += "-{}".format(version_info['prerelease'])

sha = version_info['buildmetadata']

# Force a new version (don't want to be prompted each time I parse df_files)
new_version = compat_matrix.iloc[0].copy()
new_version['OpenStudio'] = version
new_version['E+'] = "24.1.0"
new_version['SHA'] = sha
new_version['Released'] = 'TBD'
new_version['Has_Docker'] = False
new_version['Ruby'] = '3.2.2'

compat_matrix = compat_matrix.append(new_version).sort_values('OpenStudio', ascending=False).reset_index(drop=True)

** If you want to do it manually **

## Quick look at the compat matrix

In [ ]:
compat_matrix.head()

In [ ]:
compat_matrix['Has_Docker'].value_counts()

In [ ]:
# Oldest versions to have a Docker image
compat_matrix[compat_matrix['Has_Docker']].tail()

In [ ]:
# Count Number of OpenStudio versions within each E+ version
compat_matrix.groupby('E+')['OpenStudio'].count()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
compat_matrix.groupby('E+')['OpenStudio'].count().plot(kind='barh', ax=ax)
ax.set_xlim(0, compat_matrix.groupby('E+')['OpenStudio'].count().max())
ax.set_title('Number of OpenStudio version for each E+ version')
ax.set_xlabel('Number of OpenStudio Versions')
plt.show()

In [ ]:
# Export to CSV
# compat_matrix.to_csv('compat_matrix.csv')

# Fix permissions and skin down the fuelcell OSW

In [ ]:
# Skinning it down is done in the model_tests.rb now
help(regression_analysis.cleanup_bloated_osws)

The permissions stuff is done in the launch docker shell scripts

If you want to do it manually

Need to do:
    
    sudo chown -R $USER * 
    sudo find . -type f -exec chmod 664 {} \;

# Parse out.osw files

<p style='font-size: 16px; text-align: center;'><strong style='color: red;'>NOTE</strong>: This section analyzes the results of the <strong style='color: red;'>MODEL</strong> tests (`model_tests.rb`)</p>

Sections 8, 9 and 10 at the end of the notebook help in analyzing the results of the `SDD_tests.rb`, `utilities_tests.rb` and `sql_tests.rb`

In [ ]:
from imp import reload
reload(regression_analysis)

## Without custom tags

In [ ]:
df_files = regression_analysis.find_info_osws(compat_matrix=compat_matrix, testtype='model')

In [ ]:
# Limit to most recent versions so to no clutter display
df_files = df_files[['23.1.0', '23.2.0', '24.1.0']]
#df_files = df_files[['9.4.0', '9.5.0']]

## With custom Tags

In [ ]:
df_files = regression_analysis.find_info_osws_with_tags(compat_matrix=compat_matrix,
                                                        # Switch to True/False
                                                        tags_only=False,
                                                        testtype='model')

In [ ]:
max_eplus_versions = 3
df_files = df_files[
            df_files.columns.get_level_values(level='E+')
            .unique()[-max_eplus_versions:]
        ]

In [ ]:
from packaging import version
SPACE_DEFAULTED_TO_ON = '3.5.0'

def filter_on_space_enabled(df_files: pd.DataFrame, space_enabled: bool) -> pd.DataFrame:
    
    cols = []
    for col in df_files.columns:
        ep_version, os_version, space_tag = col
        v = version.parse(os_version)
        if v >= version.parse(SPACE_DEFAULTED_TO_ON):
            if space_enabled:
                if space_tag != 'WithoutSpaces':
                    cols.append(col)
            else:
                if space_tag == 'WithoutSpaces':
                    cols.append(col)
        else:
            if space_enabled:
                if space_tag == 'WithSpaces':
                    cols.append(col)
            else:
                if space_tag != 'WithSpaces':
                    cols.append(col)
    return df_files[cols]

In [ ]:
# WithSpaces
df_files = filter_on_space_enabled(df_files, True)

# WithoutSpaces
# df_files = filter_on_space_enabled(df_files, False)

In [ ]:
# Remove python tests
# df_files = df_files[df_files.index.get_level_values('Type') != 'py']

## Output the test status: Fail/Success/Blank

In [ ]:
# Prepare the dataframe 
# If you get an error, make sure the df_files is up to date by reruning section 3.1 or 3.2 above
success = regression_analysis.success_sheet(df_files)

### Entire success table

### Filter for a few tests only

#### Only those where some are missing or failed

In [ ]:
success = success.loc[success.any(axis=1)].sort_index()

In [ ]:
# filt = success[(success == '').any(axis=1) |
#                (success == 'Fail').any(axis=1)].index.get_level_values(0).unique().tolist()

filt = success['n_fail+missing']>0

headers = {
    'selector': 'th.col_heading',
    'props': 'border-style: solid; border-width: 0.5px;'
}

(
    success.loc[filt].style
    .applymap(regression_analysis.background_colors)
    .set_table_styles(regression_analysis.getStyles())
    .set_caption("Test Success")
    # .set_table_styles([headers])
)

In [ ]:
(success[success[('24.1.0', '3.8.0-rc2', 
                  'ca1c536250'
                  #'WithoutSpaces'
                 )] != 'Success'].style
          .applymap(background_colors)
          .set_table_styles(styles)
          .set_caption("Test Success"))

#(success[success[('9.2.0', '2.9.0', 'develop_merge')] != 'Success'][['9.1.0', '9.2.0']].style
#          .applymap(background_colors)
#          .set_table_styles(styles)
#          .set_caption("Test Success"))

#### Other examples

In [ ]:
# Filter on a single containing string
filt = success.index.get_level_values(0).str.contains('shadowcalculation')

# Filter on a pattern
#filt = success.index.get_level_values(0).str.match(r'(exterior_equipment)|(meters)|(plant_op_schemes)|(avms_temp)')

(success.loc[filt].style.applymap(background_colors).set_table_styles(styles)
          .set_caption("Test Success"))

In [ ]:
# Output command to rerun the tests that used to run in the previous version
# and now don't
torun = success[(success[('23.2.0', '3.7.0', '')] == 'Success') &
                (success[('24.1.0', '3.8.0-rc2', 'ca1c536250')] != 'Success')]

# s = "CUSTOMTAG=SHA /home/julien/Software/Others/OS-build/Products/openstudio-2.7.0 model_tests.rb -n '/"
s = "openstudio model_tests.rb -n '/"
tests = []
for i, (test, ext) in enumerate(torun.index.tolist()):
    test_name = "test_{}_{}".format(test, ext)
    #test_name = "test_{}".format(test)
    #s += " --name test_{}_{}".format(test, ext)
    if i < len(torun)-1:
        s+='({})|'.format(test_name)
    else:
        s+='({})'.format(test_name)
    tests.append(test_name)

s += "/'"
if tests:
    print(s)
else:
    print("No new failures")

In [ ]:
torun.style.applymap(background_colors).set_table_styles(styles)

### Export to Google

In [ ]:
spreadsheet = '/EffiBEM&NREL-Regression-Test_Status'

## Output Missing tests: ruby versus osm

In [ ]:
test_impl = regression_analysis.test_implemented_sheet(df_files=df_files, success=success,
                                   only_for_mising_osm=False)

In [ ]:
test_impl[~test_impl['osm']]

## Ouput the total_site_energy (kBTU)

In [ ]:
site_kbtu = df_files.applymap(regression_analysis.parse_total_site_energy)

## Output the rolling percent difference of total kBTU from one version to the next

In [ ]:
site_kbtu_change = site_kbtu.pct_change(axis=1)

In [ ]:
#site_kbtu_change.loc['pv_and_storage_facilityexcess']
#site_kbtu_change.loc[site_kbtu_change.index.get_level_values(0).str.contains('flat_plate')]

In [ ]:
row_threshold = 1. / 100.0
display_threshold = 0.1 / 100.0

s_last_diff = site_kbtu_change.iloc[:, -1].sort_values()
fail_mask = s_last_diff.abs() > row_threshold
failures_idx = s_last_diff.index[fail_mask]

warn_mask = s_last_diff.abs() > display_threshold
warnings_idx = s_last_diff.index[warn_mask & ~fail_mask]

nrows = (not failures_idx.empty) + (not warnings_idx.empty)

n_fail = s_last_diff[failures_idx].size
n_warn = s_last_diff[warnings_idx].size
if n_fail + n_warn == 0:
    print("Nothing to display")
else:
    if nrows > 1:
        fig, axes = plt.subplots(nrows=nrows, ncols=1, figsize=(16, n_fail + n_warn),
                                 gridspec_kw={'height_ratios': [n_fail, n_warn]})
    else:
        fig, ax = plt.subplots(nrows=nrows, ncols=1)
        axes = [ax]
    
    if not failures_idx.empty:
        s_last_diff[failures_idx].plot(kind='barh', 
                                       # figsize=(16, s_last_diff[failures_idx].size),
                                       title=f'Above {row_threshold=:.3%}', 
                                       ax=axes[0])
    else:
        print(f"No diff above {row_threshold=:.3%}")


    if not warnings_idx.empty:
        s_last_diff[warnings_idx].plot(kind='barh',
                                       # figsize=(16, s_last_diff[warnings_idx].size),
                                       title=f'Above {display_threshold=:.3%}',
                                       ax=axes[-1])
    else:
        print(f"No diff above {display_threshold=:.3%}")

    for ax in axes:
        ax.xaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:.2%}'.format(y))) 
        #vals = ax.get_xticks()
        #ax.set_xticklabels(['{:,.2%}'.format(x) for x in vals])

    fig.tight_layout()

### Heatmap > 1% change

In [ ]:
row_threshold = 1. / 100.0
display_threshold = 0.1 / 100.0

figname = 'site_kbtu_pct_change_row{}_display{}.png'.format(row_threshold, 
                                                            display_threshold)

print("Row threshold = {:.2%}, Cell Display Threshold = {:.2%}".format(row_threshold, display_threshold))

regression_analysis.heatmap_sitekbtu_pct_change(site_kbtu_change=site_kbtu_change,
                            row_threshold=row_threshold, display_threshold=display_threshold,
                            savefig=True, figname=figname,
                            show_plot=True, save_indiv_figs_for_ax=False)

### Heatmap > 0.5% change

In [ ]:
row_threshold  = 0.05 / 100.0
display_threshold=0.01 / 100.0

figname = 'site_kbtu_pct_change_row{}_display{}.png'.format(row_threshold, 
                                                            display_threshold)
print("Row threshold = {:.2%}, Cell Display Threshold = {:.2%}".format(row_threshold, display_threshold))
regression_analysis.heatmap_sitekbtu_pct_change(site_kbtu_change=site_kbtu_change,
                            row_threshold=row_threshold, display_threshold=display_threshold,
                            savefig=True, figname=figname,
                            show_plot=True, save_indiv_figs_for_ax=True)

In [ ]:
threshold = 0.1 / 100.0
NEW_EP = "23.1.0"
NEW_OS = "3.6.0"
new_diffs = site_kbtu_change.loc[site_kbtu_change[(NEW_EP, NEW_OS)].abs() >= threshold, NEW_EP]
# Sort by abs of 2.4.2
new_diffs = new_diffs.loc[new_diffs[NEW_OS].abs().sort_values(ascending=False).index]
# new_diffs.to_csv('NewDiffs.csv')
print(f"New differences that are above a threshold of {threshold:.3%}")
(new_diffs.style
          .format(lambda x: '{:.3%}'.format(x) if not np.isnan(x) else '-')
          .set_table_styles([{'selector': 'tr', 'props': [('border-style','solid'),('border-width','1px')]}]))

In [ ]:
["_".join(x) for x in new_diffs.index.tolist()]

In [ ]:
# Output command to run the ones were we have big diffs
torun = new_diffs[new_diffs[NEW_OS].abs()>= (0.1/100.0)]

s = "openstudio model_tests.rb -n '/"
tests = []
for i, (test, ext) in enumerate(torun.index.tolist()):
    if test == 'unitary_system_performance_multispeed':
        continue
    test_name = "test_{}_{}".format(test, 'osm')
    #test_name = "test_{}".format(test)
    #s += " --name test_{}_{}".format(test, ext)
    if i < len(torun)-1:
        s+='({})|'.format(test_name)
    else:
        s+='({})'.format(test_name)
    tests.append(test_name)

s += "/'"
print(s)

In [ ]:
len(new_diffs[new_diffs[NEW_OS].abs()>= (0.1/100.0)].index.tolist())

## Absolute diff

In [ ]:
diff = site_kbtu.iloc[:, 1:].div(site_kbtu.iloc[:, 0], axis=0)-1
diff = diff[(diff != 0.0).any(axis=1)]
diff = diff[diff.notnull().any(axis=1)]

In [ ]:
def style_zero(v, props=''):
    return props if v == 0 else None
diff.style.bar(align='mid', color=['#d65f5f', '#5fba7d']).applymap(style_zero, props='color:green;').format('{:.3%}')

In [ ]:
row_threshold = 1. / 100.0
display_threshold = 0.1 / 100.0

row_threshold  = 0.05 / 100.0
display_threshold=0.01 / 100.0

row_threshold = 0.005 / 100.0
display_threshold = 0.001 / 100.0

figname = 'site_kbtu_abs_pct_change_row{}_display{}.png'.format(row_threshold, 
                                                            display_threshold)

print("Row threshold = {:.3%}, Cell Display Threshold = {:.3%}".format(row_threshold, display_threshold))

regression_analysis.heatmap_sitekbtu_pct_change(site_kbtu_change=diff,
                            row_threshold=row_threshold, display_threshold=display_threshold,
                            savefig=False, figname=figname,
                            show_plot=True, save_indiv_figs_for_ax=False)

## Difference in end use

In [ ]:
# Input a threshold, here 0.5%
threshold = 0.5/100.0

over_threshold = (site_kbtu.pct_change(axis=1).abs() > threshold).sum(axis=0).to_frame()
col = 'Count (ABS(pct_diff) > {:.2%})'.format(threshold)
over_threshold.columns = [col]

In [ ]:
(over_threshold.style.set_table_styles([{'selector': 'tr', 'props': [('border-style','solid'),('border-width','1px')]}]))

In [ ]:
over_threshold.replace(0, np.nan).dropna().sort_values(col, ascending=False)

In [ ]:
version_1 = '3.1.0'
version_2 = '3.2.0'


all_diffs = {}
failed = {}
for index, row in  df_files.T.reset_index(level=0, drop=True).T.iterrows():
    diff_ok = True
    try:
        cleaned_end_use_2 = regression_analysis.parse_end_use(row[version_2])
        ok2 = True
    except:
        cleaned_end_use_2 = 'Failed'
        diff_ok = False
        ok2 = False
    try:
        cleaned_end_use_1 = regression_analysis.parse_end_use(row[version_1])
        ok1 = True
    except:
        cleaned_end_use_1 = 'Failed'
        diff_ok = False
        ok1 = False
    if diff_ok:
        pct_diff = (cleaned_end_use_2 - cleaned_end_use_1) / cleaned_end_use_1
        
        all_diffs[index] = {version_1: cleaned_end_use_1,
                            version_2: cleaned_end_use_2,
                            'diff': pct_diff}
    else:
        failed[index] = {version_1: ok1,
                         version_2: ok2}
        
df_failed = pd.DataFrame(failed).T

In [ ]:
# See the ones that changed: False means it fails, True means it worked
df_failed[df_failed[version_1] != df_failed[version_2]]

In [ ]:
max_diffs = {}
for test, d in all_diffs.items():
    #dmax = 
    max_diffs[test] = {'Max': d['diff'].max().max(),
                       'Min': d['diff'].min().min(),
                       'Total Diff': (d[version_2][('Total', 'kBtu')].sum()
                                      - d[version_1][('Total', 'kBtu')].sum()) / d[version_1][('Total', 'kBtu')].sum()}
    
    
df_diffs = pd.DataFrame(max_diffs).T

In [ ]:
df_diffs[~(df_diffs == 0).all(axis=1)].style.format("{:.5%}")

In [ ]:
df_diffs

In [ ]:
from matplotlib.ticker import FuncFormatter

In [ ]:
test = ('air_chillers', 'osm')

fig, ax = plt.subplots(figsize=(16,9))

fmt = lambda x,pos: '{:.0%}'.format(x)

sns.heatmap(all_diffs[test]['diff'].dropna(how='all', axis=0).dropna(how='all', axis=1).abs(),
            ax=ax, cmap='YlOrRd',
            vmin=0, vmax=1,
            cbar_kws={'format': mpl.ticker.FuncFormatter(fmt)},
            annot=all_diffs[test]['diff'].dropna(how='all', axis=0).dropna(how='all', axis=1), fmt='.1%')
ax.set_title("Percent difference in End Use By Fuel for test '{}.{}' between {}"
             " and {}".format(test[0], test[1], version_2, version_1))
plt.show()

# Ruby versus Python

In [ ]:
df_files = regression_analysis.find_info_osws(compat_matrix=compat_matrix, testtype='model')
df_files = df_files[['23.2.0']]
site_kbtu = df_files.applymap(regression_analysis.parse_total_site_energy)
rb_py = site_kbtu.loc[:, ('23.2.0', '3.7.0')].unstack('Type')[['rb', 'py']]
rb_py = rb_py[rb_py.any(axis=1)]
rb_py = rb_py[rb_py.diff(axis=1).abs().iloc[:, -1] > 0]
rb_py['abs_diff'] = rb_py['py'] - rb_py['rb']
rb_py['rel_diff'] = rb_py['abs_diff'] / rb_py['rb']

In [ ]:
(
    rb_py.reindex(rb_py['rel_diff'].abs().sort_values(ascending=False).index)
    .style
    .format('{:,.0f}', subset=['rb', 'py', 'abs_diff'])
    .format('{:.4%}', subset='rel_diff')
)

# Find missing tests: Map tests to Cpp classes

## Grep in ruby and osm tests

In [ ]:
os.chdir(os.path.join(ROOT_DIR, 'model/simulationtests/'))

# Grep in ruby test for Model:: statements (works on Unix only)
grep = !grep "Model::" *.rb
objs = pd.DataFrame([x.split(':', maxsplit=1 ) for x in grep], columns=['file', 'grepped_line'])

# Grep in ruby test for Model:: statements
grep_lib = !/bin/grep "Model::" ./lib/*.rb
objs_lib = pd.DataFrame(grep_lib, columns=['grepped_line'])
objs_lib['file'] = 'lib/baseline_model.rb'

# Find all Model namespace Classes by getting name from the cpp files
os_classes = !ls ~/Software/Others/OpenStudio/openstudiocore/src/model/*.cpp
os_classes = [os.path.split(os.path.splitext(p)[0])[1] for p in os_classes]

os.chdir(ROOT_DIR)

In [ ]:
model_object_pat = re.compile(r'OpenStudio::Model::(.*?)\.new')
def parse_model_object(s):
    m = model_object_pat.search(s)
    if m:
        return m.groups()[0]
    else:
        print('Cannot match {}'.format(s))
        return None
    
objs['ModelObject'] = objs['grepped_line'].apply(parse_model_object)
objs_lib['ModelObject'] = objs_lib['grepped_line'].apply(parse_model_object)

# Concat both
objs = pd.concat([objs, objs_lib])

In [ ]:
# 'AirTerminalSingleDuctUncontrolled' was deprecated (by E+) 
# and replaced with 'AirTerminalSingleDuctConstantVolumeNoReheat'
set(objs['ModelObject']) - set(os_classes) 

In [ ]:
# set(os_classes) - set(objs['ModelObject'])

In [ ]:
df_os_classes = pd.DataFrame(index=os_classes)
df_os_classes['In Ruby Test'] = False
df_os_classes = df_os_classes.join(objs.groupby('ModelObject')['file'].apply(list))
df_os_classes.loc[df_os_classes['file'].notnull(),
                  'file'] = df_os_classes.loc[df_os_classes['file'].notnull(),
                                              'file'].apply(np.unique)
df_os_classes.loc[df_os_classes['file'].notnull(), 'In Ruby Test'] = True
df_os_classes = df_os_classes.rename(columns={'file': 'files'})

In [ ]:
df_os_classes['In Ruby Test'].value_counts()

In [ ]:
df_os_classes['In Ruby Test']

In [ ]:
#df_os_classes.to_csv('Mapping_ruby_test_to_cpp_classes.csv')

## Get comments dict from the google sheet

In [ ]:
from df2gspread import gspread2df as g2d

spreadsheet = '/EffiBEM&NREL-Regression-Test_Status'
wks_name = 'Mapping_ruby_test_to_cpp_classes'

df = g2d.download(spreadsheet, wks_name, col_names = True, row_names = True)
#comments_dict = df['IsNormal'].to_dict()
comments_dict = df.loc[df['IsNormal'] != '', 'IsNormal'].to_dict()
comments_dict

In [ ]:
s = pd.Series(comments_dict)
s = s[s.str.lower().str.contains('added')].str.split(':', expand=True)[1].str.strip().sort_values()

In [ ]:
n_tot_obj = 0
n_tot_tests = 0
for index, val in s.reset_index().groupby(1)['index'].apply(list).items():
    if index == 'pv_and_storage_facilityexcess.rb':
        test = 'pv_and_storage_facilityexcess.rb and pv_and_storage_demandleveling.rb'
        n_tot_tests += 1
    else:
        test = index
    n_tot_tests += 1
    n_tot_obj += len(val)
    print("**{}** ({})".format(test, len(val)))
    print()
    for x in val:
        print("* {}".format(x))
    print("\n")
print("\n**Total Added: {} objects in {} tests**".format(n_tot_obj, n_tot_tests))

In [ ]:
#comments.set_index('Test')['IsNormal'].to_dict()

In [ ]:
# Merge comments
comments = pd.Series(comments_dict, name='IsNormal')
df_os_classes = df_os_classes.join(comments)
df_os_classes = df_os_classes[['In Ruby Test', 'IsNormal', 'files']]

## Find objects in the osm tests

In [ ]:
# change dir to the model test directory
os.chdir(os.path.join(ROOT_DIR, 'model/simulationtests/'))

# Compile a regex
os_class_pattern = re.compile(r'OS:(.*?),')

# Initialize a column of empty lists
df_os_classes['osms'] = np.empty((len(df_os_classes), 0)).tolist()

# Loop on all osms, and find OS objects
for osm_path in gb.glob('*.osm'):
    with open(osm_path) as f:
        lines = f.readlines()
    for line in lines:
        m = os_class_pattern.match(line)
        if m:
            classname = m.groups()[0].replace(':','')
            # Deprecated by E+
            if classname == 'AirTerminalSingleDuctUncontrolled':
                classname = 'AirTerminalSingleDuctConstantVolumeNoReheat'
            if classname in df_os_classes.index:
                # Append osm_path to list if not already in it
                if not osm_path in df_os_classes.loc[classname, 'osms']:
                    df_os_classes.loc[classname, 'osms'].append(osm_path)
            else:
                print("Cannot find {} in df_os_classes".format(classname))
                
os.chdir(ROOT_DIR)

In [ ]:
df_os_classes.loc[df_os_classes['osms'].apply(len) == 0, 'osms'] = None

In [ ]:
filt1 = ~df_os_classes['In Ruby Test']
filt2 = df_os_classes['IsNormal'].isnull()
filt3 = df_os_classes['osms'].isnull()
df_os_classes[filt1 & filt2 & filt3] # .apply(lambda x: print(x.name), axis=1)

In [ ]:
in_ruby = df_os_classes['In Ruby Test']
in_ruby.value_counts()

In [ ]:
is_normal = df_os_classes['IsNormal'].notnull()

In [ ]:
in_osm = df_os_classes['osms'].notnull()

In [ ]:
(in_ruby | is_normal).value_counts()

In [ ]:
(in_ruby | is_normal | in_osm).value_counts()

In [ ]:
df_os_classes.fillna('')

## Upload to Google

# Test convergence of OSMs

## Running the same in.OSW

In [ ]:
RUN_N_TIMES = 5
START_AT=5
OS_CLI='openstudio'
# OS_CLI='ruby'
OS_CLI = '/home/julien/Software/Others/OS-build/Products/openstudio'
# Windows path should be like:
# OS_CLI = r'C:\openstudio-2.5.0\bin\openstudio'

In [ ]:
import subprocess
from shutil import copyfile

# Modify to suit your needs. You should have run the model_tests.rb with this
# test first, so that the in.osw etc exists first...
os.chdir(os.path.join(ROOT_DIR, 'testruns/evaporative_cooling.osm/'))

r = {}
o = {}
e = {}
for i in range(START_AT, START_AT + RUN_N_TIMES):
    process = subprocess.Popen([OS_CLI, 'run', '-w', 'in.osw'], shell=False,
                           stdout=subprocess.PIPE, 
                           stderr=subprocess.PIPE)

    # wait for the process to terminate
    out, err = process.communicate()
    o[i] = out
    e[i] = err
    errcode = process.returncode
    r[i] = regression_analysis.parse_total_site_energy('out.osw')
    print("{} - {:,.0f}".format(i, r[i]))
    # Copy idf file
    idf_path = os.path.abspath('./run/in.idf')
    copyfile(idf_path, os.path.abspath('./run_{}.idf'.format(i)))
    
# Say to user
if regression_analysis.platform.system() == 'Linux':
    !echo "THIS IS DONE" | espeak

os.chdir(ROOT_DIR)

In [ ]:
try:
    result = pd.concat([result, pd.Series(r)])
except:
    result = pd.Series(r)

In [ ]:
result

In [ ]:
result.describe()

In [ ]:
sns.boxplot(result)

In [ ]:
result.plot(kind='bar')

In [ ]:
ax = ((result - result.iloc[0]) / result.iloc[0]).plot(kind='bar')
def fmt(x, pos): return '{:.0%}'.format(x)

ax.yaxis.set_major_formatter(mpl.ticker.FuncFormatter(fmt))
ax.set_title('% change compared to first run')

# Compare With Custom Tags

## Run the Sim Tests N Times with Custom Tags

In [ ]:
# PLATFORM = 'Ubuntu'
PLATFORM = None

N = 5
START_AT = 1
# Override if you have only run N times before, and do not want to override
# START_AT = 10

# Save idf file next to out.osw? 
# Useful for checking IDF diffs for ruby tests that are unstable
SAVE_IDF=False

# Filter to pass to model_tests.rb. Input NONE for all tests
REGRESSION_TEST_FILTER = None

# Override examples:
# REGRESSION_TEST_FILTER = '(test_absorption_chillers_rb)|(test_additional_props_rb)|(test_air_chillers_rb)|(test_air_terminals_rb)|(test_airloop_and_zonehvac_rb)|(test_airterminal_cooledbeam_rb)|(test_autosize_hvac_rb)|(test_centralheatpumpsystem_rb)|(test_coolingtowers_rb)|(test_dist_ht_cl_rb)|(test_dsn_oa_w_ideal_loads_rb)|(test_dual_duct_rb)|(test_ducts_and_pipes_rb)|(test_ems_rb)|(test_evaporative_cooling_rb)|(test_exterior_equipment_rb)|(test_fan_on_off_rb)|(test_fuelcell_rb)|(test_generator_microturbine_rb)|(test_headered_pumps_rb)|(test_hightemprad_rb)|(test_humidity_control_rb)|(test_interior_partitions_rb)|(test_lowtemprad_constflow_rb)|(test_lowtemprad_electric_rb)|(test_lowtemprad_varflow_rb)|(test_multi_stage_rb)|(test_plant_op_deltatemp_schemes_rb)|(test_plantloop_avms_temp_rb)|(test_plenums_rb)|(test_pv_and_storage_demandleveling_rb)|(test_pv_and_storage_facilityexcess_rb)|(test_roof_vegetation_rb)|(test_solar_collector_flat_plate_water_rb)|(test_space_load_instances_rb)|(test_surface_properties_rb)|(test_unitary_system_performance_multispeed_rb)|(test_vrf_rb)|(test_water_heaters_rb)|(test_zone_control_contaminant_controller_rb)|(test_zone_fan_exhaust_rb)'
# REGRESSION_TEST_FILTER = 'centralheatpumpsystem_osm'
# REGRESSION_TEST_FILTER = 'fourpipebeam'
# REGRESSION_TEST_FILTER = 'somethingthatdoesntexistfortesting'

# If you need to hardset the E+ executable path, otherwise leave as None
ENERGYPLUS_EXE_PATH = None
# ENERGYPLUS_EXE_PATH = '/home/julien/Software/Others/OS-build/EnergyPlus-8.9.0-40101eaafd-Linux-x86_64/EnergyPlus-8-9-0/energyplus-8.9.0'

# Path to your cli, if use system ruby make sure it's setup correctly via openstudio.rb
# If you pass None, defaults to 'openstudio' (has to be in PATH)
OS_CLI = None
# OS_CLI = '/home/julien/Software/Others/OS-build2/Products/openstudio-2.4.3'
# OS_CLI = 'ruby'
# Windows path should be like this (forward slashes work both in linux and windows)
# OS_CLI = 'C:/openstudio-2.5.0/bin/openstudio'
# or (notice the 'r' denoting a raw string)
# OS_CLI = r'C:\openstudio-2.5.0\bin\openstudio'
# You can test this
# regression_analysis.test_os_cli(OS_CLI)

In [ ]:
regression_analysis.test_stability(os_cli=OS_CLI, test_filter=REGRESSION_TEST_FILTER,
               run_n_times=N, start_at=START_AT,
               save_idf=SAVE_IDF,
               energyplus_exe_path=ENERGYPLUS_EXE_PATH,
               platform_name=PLATFORM)

## Load the Custom-tagged files

In [ ]:
df_files = regression_analysis.find_info_osws_with_tags(compat_matrix=None,
                                                        tags_only=True)


In [ ]:
success = regression_analysis.success_sheet(df_files)

In [ ]:
filt = success['n_fail+missing']>0

(success.loc[filt].style
          .applymap(background_colors)
          .set_table_styles(styles)
          .set_caption("Test Success"))

## First filter only tests that have some variations in site kBTU

I check for tests where the min accross runs isn't equal to the max

In [ ]:
site_kbtu = df_files.applymap(regression_analysis.parse_total_site_energy)

In [ ]:
site_kbtu

In [ ]:
regression_analysis.heatmap_sitekbtu_pct_change(site_kbtu=site_kbtu, row_threshold=0)

In [ ]:
# Restrict to our version of interest, drop rows with all nan
site_kbtu_this_version = site_kbtu['8.9.0']['2.4.4'].dropna(how='all')

# Keep only the custom tagged ones
# site_kbtu_this_version = site_kbtu_this_version[[x for x in site_kbtu_this_version.columns if x in keep_only_runs]]

# Filter on rows where the min is not the max
site_kbtu_this_version = site_kbtu_this_version[site_kbtu_this_version.apply(lambda row: min(row) != max(row), axis=1)]

# Make a multiindex 
site_kbtu_this_version.columns = pd.MultiIndex.from_tuples([x.split('_') for x in site_kbtu_this_version.columns],
                                                 names=['Platform', 'Run'])

For these tests where we have variations, we can visualize the deviation each run Platform/run using a boxplox:

In [ ]:
fig, ax = plt.subplots(figsize=(16,9))
site_kbtu_this_version.boxplot(ax=ax, grid=False)
ax.set_title('Boxplot of tests that have variations, by platform and run')
ax.set_ylabel('Total site kBTU')
plt.show()

## Check biggest differences by looking at CV

Second, I calculate the coefficient of variation ($CV$) for each test = standard deviation ($\sigma$) divided by mean ($\mu$)

$$CV = \frac{\sigma}{\mu}$$

I then use a set tolerance to filter out tests that have a CV that isn't above or equal to the tolerance.

In [ ]:
# Coefficient of variation: standard deviation divided by mean
cv = site_kbtu_this_version.std(axis=1) / site_kbtu_this_version.mean(axis=1)
cv.name = 'Coefficient of Variation'
cv = cv.dropna()

In [ ]:
cv.sort_values(ascending=False).to_frame().style.format('{:.5%}')

In [ ]:
cv_tol = 0.00001
print("Setting CV Tolerance to {:.3%}".format(cv_tol))

In [ ]:
sns.set_style('white')
sns.set_palette('Set2')

In [ ]:
site_kbtu_this_version.loc[('centralheatpumpsystem', 'rb')].max()

In [ ]:
site_kbtu_this_version.loc[('centralheatpumpsystem', 'rb')].min()

In [ ]:
# ax = cv[cv >= cv_tol].sort_values(ascending=True).plot(kind='barh', figsize=(16,9))
# Or Plot all

toplot = cv.sort_values(ascending=True)
toplot.index = ["_".join(x) for x in toplot.index]

ax = toplot.plot(kind='barh', figsize=(16,9))
vals = ax.get_xticks()
ax.set_xticklabels(['{:3.4f}%'.format(x*100) for x in vals])
ax.set_title('Coefficient of Variation for tests that are above cv_tol={:.3%}'.format(cv_tol))

for i, rect in enumerate(ax.patches):
    label = ax.get_yticklabels()[i].get_text() 
    val = toplot[i]
    ax.annotate("{:.5%}".format(val), 
                xy=(rect.get_x()+rect.get_width(), rect.get_y()+rect.get_height()/2))

plt.show()

**Using the same tests, we can visualize the total site kBTU for each:**

In [ ]:
fig, ax = plt.subplots()
site_kbtu_this_version.reindex(index=cv[cv >= cv_tol].index).plot(kind='bar', ax=ax)
ax.set_title('Total site kBTU for tests that are above the CV tolerance')
ax.set_ylabel('Total Site kBTU')
plt.show()

<p style="font-size: 40px; color:red;">ANYTHING PAST THIS POINT NEEDS CLEANING</p>

## Could Ruby test just be unstable regardless of platform?

1) First,  **the big differences are in the ruby tests mostly** (except 2.). I've mentionned already that I fixed a bunch of instabilities in the ruby tests, but there are some I couldn't fix yet: **could the ruby tests in question just be unstable regardless of platform?**

I plot the entire heatmap (all OS version) of these tests which have a CV >= cv_tol:

In [ ]:
toplot = site_kbtu.reindex(index=cv[cv >= cv_tol].index)
toplot = toplot[[x for x in toplot.columns if x[2] in ([''] + keep_only_runs)]]

In [ ]:
regression_analysis.heatmap_sitekbtu_pct_change(site_kbtu=toplot,
                            row_threshold=0.0000, display_threshold=0.0001, 
                            savefig=False, show_plot=True, figsize=(16,6))

The following tests are unstable regardless of platform:
    
* airloop_and_zonehvac.rb
* evaporative_cooling.rb
* surface_properties.rb 
* unitary_system_performance_multispeed.rb (edited)

The big unknown is **what the heck happened in Windows Run 1 for unitary_vav_bypass**?

## One OSM test produces different results on different platform

----

2) One very important exception to (1) above is the `evaporative_cooling.osm` test: **seems to be stable on both platform, but it doesn't have the same numbers on Ubuntu versus windows! Further investigation is warranted.**

Note: You might say it's hard to tell if the OSM is stable on a given platform with two runs. I ran it 10 times on Ubuntu, and it is stable.

    count    1.000000e+01
    mean     7.632714e+06
    std      9.817002e-10
    min      7.632714e+06
    25%      7.632714e+06
    50%      7.632714e+06
    75%      7.632714e+06
    max      7.632714e+06
    dtype: float64

## Run N more times on a given machine to have more info

In [ ]:
torun

In [ ]:
s = "'/"

torun = (cv[cv >= cv_tol].index.tolist())

for i, (test, ext) in enumerate(torun):
    test_name = "test_{}_{}".format(test, ext)
    #test_name = "test_{}".format(test)
    #s += " --name test_{}_{}".format(test, ext)
    if i < len(torun)-1:
        s+='({})|'.format(test_name)
    else:
        s+='({})'.format(test_name)

s += "/'"
print("ruby model_tests.rb -n {}".format(s))

## Reload with more runs

In [ ]:
df_files = regression_analysis.find_info_osws_with_tags(compat_matrix=compat_matrix)
subset_files = df_files[[x for x in df_files.columns
                         if x[1] == '2.4.1' 
                         #and x[2] != 'Ubuntu_run2'
                        ]]
# Keep only those that I run more than twice
subset_files = subset_files.loc[subset_files[('8.8.0', '2.4.1', 'Ubuntu_run3')].notnull()]

# Parse site_kbtu
site_kbtu = subset_files.applymap(regression_analysis.parse_total_site_energy)

# Restrict to our version of interest, drop rows with all nan
site_kbtu_241 = site_kbtu['8.8.0']['2.4.1'].dropna(how='all')

# Keep only the custom tagged ones
site_kbtu_241 = site_kbtu_241[[x for x in site_kbtu_241.columns if x != '']]

# Make a multiindex 
site_kbtu_241.columns = pd.MultiIndex.from_tuples([x.split('_') for x in site_kbtu_241.columns],
                                                 names=['Platform', 'Run'])

In [ ]:
site_kbtu_241.groupby(level='Platform', axis=1).mean().plot(kind='bar')

In [ ]:
def heatmap_from_pct_diff(toplot, display_threshold=0.001, 
                          title=None):
    # Prepare two custom cmaps with one single color
    grey_cmap = mpl.colors.ListedColormap('#f7f7f7')
    green_cmap = mpl.colors.ListedColormap('#f0f7d9')


    w = 16
    h = w * toplot.shape[0] / (3 * toplot.shape[1])

    fig, ax = plt.subplots(figsize=(w, h))

    # Reserve 1.5 inches at bottom for explanation
    #fig.subplots_adjust(bottom=1.5/h)

    # Same as: fmt = lambda x,pos: '{:.1%}'.format(x)
    def fmt(x, pos): return '{:.1%}'.format(x)


    # Plot with colors, for those that are above the display_threshold
    sns.heatmap(toplot.abs(), mask=toplot.abs() <= display_threshold,
                ax=ax, cmap='YlOrRd',  # cmap='Reds', 'RdYlGn_r'
                vmin=0, vmax=0.5,
                cbar_kws={'format': mpl.ticker.FuncFormatter(fmt)},
                annot=toplot, fmt='.2%', linewidths=.5)

    # Plot a second heatmap on top, only for those that are below
    sns.heatmap(toplot, mask=((toplot.abs() > display_threshold) |
                              (toplot.abs() == 0)),
                cbar=False,
                annot=True, fmt=".4%", annot_kws={"style": "italic"},
                ax=ax, cmap=grey_cmap)

    # Plot a third heatmap on top, only for those that are zero,
    # no annot just green
    sns.heatmap(toplot, mask=(toplot.abs() != 0),
                cbar=False,  # linewidths=.5, linecolor='#cecccc',
                annot=False,
                ax=ax, cmap=green_cmap)

    if title:
        ax.set_title(title)

    plt.show()


In [ ]:
# % from the mean siteKBTU of the test
toplot = ((site_kbtu_241.T - site_kbtu_241.T.mean())/(site_kbtu_241.T.mean())).T

heatmap_from_pct_diff(toplot, title='Percentage difference from mean of test (both_versions)',
                     display_threshold=0.001)

In [ ]:
# % from the mean siteKBTU of the test
mean_ubuntu = site_kbtu_241['Ubuntu'].mean(axis=1)
toplot = ((site_kbtu_241.T - mean_ubuntu)/(mean_ubuntu)).T

heatmap_from_pct_diff(toplot, title='Percentage difference from mean of test for Ubuntu platform',
                     display_threshold=0.0001)

# Miscellaneous helper scripts

## Delete previous runs

In [ ]:
files = gb.glob('./test/plant_op_schemes_*')
files

In [ ]:
for f in files:
    os.remove(f)

## Look at one cleaned out.osw

In [ ]:
!ls test/*generator*

In [ ]:
data = regression_analysis.load_osw('test/generator_microturbine.rb_2.0.4_out.osw')

In [ ]:
data

## Rename out.oswS

## Zip custom tagged files for sharing

In [ ]:
# Copy all tagged runs into a 'Tagged' directory for manual zipping
os.mkdir('test/Tagged')
for f in gb.glob('test/*out_*.osw'):
    print(f)
    dst_path  = f.replace('test/', 'test/Tagged/')
    copyfile(f, dst_path)
    

In [ ]:
# Zip in one go...
import zipfile
import glob as gb
with zipfile.ZipFile("Tagged.zip", "w") as z:
    for f in gb.glob('test/*out_*.osw'):
        z.write(f)

In [ ]:
# Verify it worked
z = zipfile.ZipFile("Tagged.zip")
z.printdir()
z.close()

# Analyzing SDD tests

In [ ]:
TEST_DIR = os.path.join(ROOT_DIR, 'test')

SDD_TEST_DIR = os.path.join(ROOT_DIR, 'testruns/SddReverseTranslator/')
SDD_SIM_XML_DIR = os.path.join(ROOT_DIR, 'model/sddtests/')
MODEL_TEST_DIR = os.path.join(ROOT_DIR, 'model/simulationtests/')

In [ ]:
def escapeName(filename):
    return (filename.replace('-','_')
                    .replace('+','_')
                    .replace(' ','_')
                    .replace("=",'_'))

## Generate tests for SDD RT

In [ ]:
all_tests = [os.path.relpath(x, SDD_SIM_XML_DIR) for x in gb.glob(os.path.join(SDD_SIM_XML_DIR, '*.xml'))]
all_tests = sorted(all_tests)
for t in all_tests:
    filename = os.path.basename(t).replace(' - ap.xml', '')
    print("  def test_RT_{}".format(escapeName(filename)))
    print("    sdd_rt_test('{}')".format(t))
    print("  end")
    print("")

## Generate tests for SDD FT

In [ ]:
all_model_paths = gb.glob('model/simulationtests/*.osm')
all_model_filenames = [ os.path.basename(p) for p in all_model_paths]
all_model_filenames = sorted(all_model_filenames)

In [ ]:
for filename in all_model_filenames:
    print("  def test_FT_{}".format(escapeName(filename.replace('.osm', ''))))
    print("    sdd_ft_test('{}')".format(filename))
    print("  end")
    print("")

## SDD Reverse Translator

In [ ]:
df_files = regression_analysis.find_info_osws(compat_matrix=compat_matrix, testtype='sddrt')

In [ ]:
# Prepare the dataframe 
# If you get an error, make sure the df_files is up to date by reruning section 3.1 or 3.2 above
success = regression_analysis.success_sheet(df_files)

In [ ]:
filt = (success['n_fail+missing'] > 0)
n_failed = filt.sum()
print("There are {} test(s) that failed at least once".format(n_failed))

last_version = success.columns[-4]
n_failed_last_version = (success.iloc[:, -4] != 'Success').sum()
print("In the last version ran {}, {} tests failed".format(last_version, n_failed_last_version))

(success.loc[filt].style
          .applymap(regression_analysis.background_colors)
          .set_table_styles(regression_analysis.getStyles())
          .set_caption("Test Success"))

In [ ]:
# Output command to rerun the tests that used to run in the previous version
# and now don't
torun = success[(success[('9.3.0', '3.0.1')] == 'Success') &
                (success[('9.4.0', '3.1.0')] != 'Success')]

# s = "CUSTOMTAG=SHA /home/julien/Software/Others/OS-build/Products/openstudio-2.7.0 model_tests.rb -n '/"
s = "openstudio SDD_tests.rb -n '/"
tests = []
for i, (test, ext) in enumerate(torun.index.tolist()):
    test_name = "test_RT_{}".format(test)
    #test_name = "test_{}".format(test)
    #s += " --name test_{}_{}".format(test, ext)
    if i < len(torun)-1:
        s+='({})|'.format(test_name)
    else:
        s+='({})'.format(test_name)
    tests.append(test_name)

s += "/'"
print(s)

In [ ]:
site_kbtu = df_files.applymap(regression_analysis.parse_total_site_energy)
site_kbtu_change = site_kbtu.pct_change(axis=1)

### Heatmap > 1% change

In [ ]:
row_threshold = 1. / 100.0
display_threshold = 0.1 / 100.0

figname = 'site_kbtu_pct_change_row{}_display{}.png'.format(row_threshold, 
                                                            display_threshold)

print("Row threshold = {:.2%}, Cell Display Threshold = {:.2%}".format(row_threshold, display_threshold))

regression_analysis.heatmap_sitekbtu_pct_change(site_kbtu=site_kbtu,
                            row_threshold=row_threshold, display_threshold=display_threshold,
                            savefig=True, figname=figname,
                            show_plot=True, save_indiv_figs_for_ax=False)

### Heatmap > 0.5% change

In [ ]:
row_threshold  = 0.05 / 100.0
display_threshold=0.01 / 100.0

figname = 'site_kbtu_pct_change_row{}_display{}.png'.format(row_threshold, 
                                                            display_threshold)
print("Row threshold = {:.2%}, Cell Display Threshold = {:.2%}".format(row_threshold, display_threshold))
regression_analysis.heatmap_sitekbtu_pct_change(site_kbtu=site_kbtu,
                            row_threshold=row_threshold, display_threshold=display_threshold,
                            savefig=True, figname=figname,
                            show_plot=True, save_indiv_figs_for_ax=True)

## SDD Forward Translator

In [ ]:
from imp import reload
reload(regression_analysis)

In [ ]:
# Rest assured that this will find the XMLs (not OSWs)
df_files = regression_analysis.find_info_osws(compat_matrix=compat_matrix, testtype='sddft')

In [ ]:
success = regression_analysis.success_sheet_sddft(df_files)

filt = (success['n_fail'] > 0)
n_failed = filt.sum()
print("There are {} test(s) that failed".format(n_failed))

last_version = success.columns[-3]
n_failed_last_version = (success.iloc[:, -3] != 'Success').sum()
print("In the last version ran {}, {} tests failed".format(last_version, n_failed_last_version))

(success.loc[filt].style
         .applymap(regression_analysis.background_colors)
         .set_table_styles(regression_analysis.getStyles())
         .set_caption("SDD FT: Test Success")) 

In [ ]:
df_diff = regression_analysis.diff_all_xmls(df_files)

In [ ]:
df_diff.style

# SQL Test

In [ ]:
from imp import reload
reload(regression_analysis)

In [ ]:
# Rest assured that this will find the .sqltest files (not OSWs)
df_files = regression_analysis.find_info_osws(compat_matrix=compat_matrix, testtype='sql')

In [ ]:
success = regression_analysis.success_sheet_sql(df_files)

In [ ]:
filt = (success['n_fail'] > 0)
n_failed = filt.sum()
print("There are {} test(s) that failed at least once".format(n_failed))

last_version = success.columns[-2]
n_failed_last_version = (success.iloc[:, -2] == 'Fail').sum()
print("In the last version ran {}, {} tests failed".format(last_version, n_failed_last_version))

(success
# .loc[filt]
 .style
 .applymap(regression_analysis.background_colors)
 .set_table_styles(regression_analysis.getStyles())
 .set_caption("sql_tests: Test Success"))

# Utilities Test

In [ ]:
from imp import reload
reload(regression_analysis)

In [ ]:
# Rest assured that this will find the .status files (not OSWs)
df_files = regression_analysis.find_info_osws(compat_matrix=compat_matrix, testtype='utilities')

success = regression_analysis.success_sheet_utilities(df_files)


filt = (success['n_fail'] > 0)
n_failed = filt.sum()
print("There are {} test(s) that failed at least once".format(n_failed))

last_version = success.columns[-2]
n_failed_last_version = (success.iloc[:, -2] == 'Fail').sum()
print("In the last version ran {}, {} tests failed".format(last_version, n_failed_last_version))

(success
# .loc[filt]
 .style
 .applymap(regression_analysis.background_colors)
 .set_table_styles(regression_analysis.getStyles())
 .set_caption("utilities_tests: Test Success"))

In [ ]:
regression_analysis.encoding_sheet_utilities(df_files)